## Sentinel-2 vegetation mask based on NDVI and BSI

### Service Definition

This service among other informations, generates *NDVI Values* and *BSI Values*

In [80]:
service = dict([('title', 'Sentinel-2 vegetation mask based on NDVI and BSI'),
                ('abstract', 'Sentinel-2 vegetation mask based on NDVI and BSI'),
                ('id', 'ewf-s2-vegetation-mask')])

### Parameter Definition

In [4]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                   ('value', '0.3'),
                   ('title', 'NDVI threshold for the mask expression'),
                   ('abstract', 'NDVI threshold for the mask expression'),
                   ('maxOccurs', '1')])

In [5]:
bsi_threshold = dict([('id', 'bsi_threshold'),
                   ('value', '0'),
                   ('title', 'BSI threshold'),
                   ('abstract', 'BSI threshold'),
                   ('maxOccurs', '1')])

In [6]:
aoi = dict([('id', 'aoi'),
              ('title', 'Area of interest'),
              ('abstract', 'Area of interest'),
              ('value', '')])

In [7]:
username = dict([('id', '_T2Username'),
              ('title', 'Ellip username'),
              ('abstract', 'Ellip username'),
              ('value', '')])

In [8]:
api_key = dict([('id', '_T2ApiKey'),
              ('title', 'Ellip API key for data pipeline'),
              ('abstract', 'Ellip API key for data pipeline'),
              ('value', '')])

### <a name="runtime">Runtime parameter definition

**Input reference**

This is the Sentinel-2 catalogue entry URLs

In [9]:
input_references = []

### Workflow

#### Import the packages required for processing the data

In [38]:
import os
import sys

sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())
from helpers import *
%load_ext autoreload
%autoreload 2


sys.path.append('/opt/OTB/lib/python')
sys.path.append('/opt/OTB/lib/libfftw3.so.3')
os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB/lib/otb/applications'
os.environ['LD_LIBRARY_PATH'] = '/opt/OTB/lib'
os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB/lib/otb/applications'
os.environ['GDAL_DATA'] = '/opt/anaconda/share/gdal/'
import otbApplication
import gdal

gdal.UseExceptions()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
product_metadata = get_product_metadata(input_references, 
                                        username['value'],
                                        api_key['value'])

In [12]:
product_metadata

,cc,enclosure,enddate,identifier,platform,self,startdate,track,wkt,geometry
0,6.439197,https://store.terradue.com/ard-s2-boa-reflecta...,2018-08-13 14:57:19.024,71BB2847F91DABBA3EDD61668761DF53FA2080BF,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2018-08-13 14:57:19.024,039,"POLYGON((-69.9212607569422 -13.6568683838856,-...","POLYGON ((-69.9212607569422 -13.6568683838856,..."
1,1.036652,https://store.terradue.com/ard-s2-boa-reflecta...,2018-08-13 14:57:19.024,8478474798720351C60716A682EBA53EA6A6A8E9,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2018-08-13 14:57:19.024,039,"POLYGON((-70.835297389016 -12.7511412207275,-6...",POLYGON ((-70.83529738901601 -12.7511412207275...
2,0.099581,https://store.terradue.com/ard-s2-boa-reflecta...,2018-08-13 14:57:19.024,CF8EF52646BB41EF2EDE66D2E2AB8AD9320CB400,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2018-08-13 14:57:19.024,039,"POLYGON((-69.9181391041721 -12.7524314684136,-...","POLYGON ((-69.9181391041721 -12.7524314684136,..."
3,15.017480,https://store.terradue.com/ard-s2-boa-reflecta...,2018-08-13 14:57:19.024,D3B736734067194663D8A82446273A9A9FA3557C,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2018-08-13 14:57:19.024,039,"POLYGON((-70.8415341437528 -13.6554833185667,-...","POLYGON ((-70.8415341437528 -13.6554833185667,..."
4,1.339872,https://store.terradue.com/ard-s2-boa-reflecta...,2018-08-10 14:47:29.024,5CC8B39C8B44649589DAF902DA35C40153050D24,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2018-08-10 14:47:29.024,139,"POLYGON((-69.9181391041721 -12.7524314684136,-...","POLYGON ((-69.9181391041721 -12.7524314684136,..."
5,12.614442,https://store.terradue.com/ard-s2-boa-reflecta...,2018-08-10 14:47:29.024,B9D901D6B238C568B149D4884DB431F87A1FE810,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2018-08-10 14:47:29.024,139,"POLYGON((-69.9212607569422 -13.6568683838856,-...","POLYGON ((-69.9212607569422 -13.6568683838856,..."


In [13]:
min_date = min(product_metadata['startdate'])
max_date = max(product_metadata['enddate'])

#### Area of interest

In [14]:
x_min, y_min, x_max, y_max = [float(c) for c in aoi['value'].split(',')]

#### Input vsi URLs

Use the Sentinel-2 ARD using GDAL Virtual File System

In [15]:
daily_products = []

for track in product_metadata.track.unique():

    print(track)
    input_references = list(product_metadata[product_metadata.track == track].self.values)
    
    print(input_references)

    output_name = str(track)
    
    pre_proces(input_references, 
               [x_min, y_min, x_max, y_max],
               ndvi_threshold['value'],
               bsi_threshold['value'], 
               output_name, 
               username['value'], 
               api_key['value'])
    
    daily_products.append(output_name)

039
['https://catalog.terradue.com/ard-s2-boa-reflectances/search?format=atom&uid=71BB2847F91DABBA3EDD61668761DF53FA2080BF', 'https://catalog.terradue.com/ard-s2-boa-reflectances/search?format=atom&uid=8478474798720351C60716A682EBA53EA6A6A8E9', 'https://catalog.terradue.com/ard-s2-boa-reflectances/search?format=atom&uid=CF8EF52646BB41EF2EDE66D2E2AB8AD9320CB400', 'https://catalog.terradue.com/ard-s2-boa-reflectances/search?format=atom&uid=D3B736734067194663D8A82446273A9A9FA3557C']
1 B01
2 B02
3 B03
4 B04
5 B05
6 B06
7 B07
8 B08
9 B8A
10 B09
11 B11
12 B12
13 SCL
14 MSK_CLDPRB
15 MSK_SNWPRB
139
['https://catalog.terradue.com/ard-s2-boa-reflectances/search?format=atom&uid=5CC8B39C8B44649589DAF902DA35C40153050D24', 'https://catalog.terradue.com/ard-s2-boa-reflectances/search?format=atom&uid=B9D901D6B238C568B149D4884DB431F87A1FE810']
1 B01
2 B02
3 B03
4 B04
5 B05
6 B06
7 B07
8 B08
9 B8A
10 B09
11 B11
12 B12
13 SCL
14 MSK_CLDPRB
15 MSK_SNWPRB


In [69]:

prefix = ['VEGETATION_MASK',
                  'BSI_VALUES',
                  'BSI_MASK',
                  'NDVI_VALUES',
                  'NDVI_MASK',
                  'MASK']
temp_files=[]
date_format = '%Y%m%dT%H%m%S'

for index in range(len(prefix)):

    
    if prefix[index] in ['NDVI_VALUES','BSI_VALUES']:
        expression = get_mosaic_expressionx(128, len(product_metadata.track.unique()))
    else:
            
        expression = get_mask_expressionx(128, len(product_metadata.track.unique()),
                                          [10, 20] if prefix[index]=='VEGETATION_MASK' else [0])
    
    print expression
    
    
    output_name = '{0}_{1}_{2}.tif'.format(prefix[index], 
                                           min_date.strftime(date_format),
                                           max_date.strftime(date_format))
    
    daily_products = ['{}_{}.tif'.format(index, track) for track in product_metadata.track.unique()] 
    
    
    print output_name
    print daily_products
    
    
    BandMathX = otbApplication.Registry.CreateApplication('BandMathX')

    BandMathX.SetParameterStringList('il', daily_products)
    BandMathX.SetParameterString('out', 'temp_{0}'.format(output_name))
    BandMathX.SetParameterString('exp', expression)
    BandMathX.SetParameterOutputImagePixelType('out', 
                                               otbApplication.ImagePixelType_double if prefix[index] in ['NDVI_VALUES','BSI_VALUES'] else otbApplication.ImagePixelType_uint8)

    BandMathX.ExecuteAndWriteOutput()
    
    temp_files.append('temp_{0}'.format(output_name))
    
#Remove temporary files
    for prd in daily_products:
        os.remove(prd)
        os.remove(prd + '.aux.xml')
    
    



output_name='VEGETATION_MASK_BSI_NDVI-{0}-{1}.tif'. format(min_date.strftime(date_format),
                                           max_date.strftime(date_format))
#Merge the files together
merge_expressions = ['im{}b1'.format(index+1) for index in range(len(temp_files))]
merge = otbApplication.Registry.CreateApplication('BandMathX')

merge.SetParameterStringList('il', temp_files)
merge.SetParameterString('out', 'temp_{0}'.format(output_name))
merge.SetParameterString('exp',';'.join(merge_expressions))
merge.ExecuteAndWriteOutput()
    


cog('temp_{0}'.format(output_name),
        output_name)



date_format = '%Y-%m-%dT%H:%m:%SZ'    
with open(output_name.replace('.tif', '.properties'), 'wb') as file:
        
        file.write('title=GEOTIFF Vegetation Mask {0} {1} \n'.format(min_date.strftime(date_format),
                                           max_date.strftime(date_format)))
        file.write('date={0}/{1}\n'.format(min_date.strftime(date_format),
                                           max_date.strftime(date_format)))
        file.write('description={0} contains 6 bands:VEGETATION_MASK,BSI_VALUES,BSI_MASK,NDVI_VALUES,NDVI_MASK,MASK,\n'.format(output_name))
        
        file.write('geometry={0}'.format(get_image_wkt(output_name)))
#Remove temporary files        
for prd in temp_files:
    os.remove(prd)

(im1b1 == 128 && im2b1 == 128) ? 128 : (im1b1 == 10 || im2b1 == 10) ? 10 : (im1b1 == 20 || im2b1 == 20) ? 20 : 255
VEGETATION_MASK_20180810T140829_20180813T140819.tif
['0_039.tif', '0_139.tif']
(im1b1 == 128 && im2b1 == 128) ? 128 : (im1b1 >= -1 && im1b1 <= 1) ? im1b1 : (im2b1 >= -1 && im2b1 <= 1) ? im2b1 : 255
BSI_VALUES_20180810T140829_20180813T140819.tif
['1_039.tif', '1_139.tif']
(im1b1 == 128 && im2b1 == 128) ? 128 : (im1b1 == 0 || im2b1 == 0) ? 0 : 255
BSI_MASK_20180810T140829_20180813T140819.tif
['2_039.tif', '2_139.tif']
(im1b1 == 128 && im2b1 == 128) ? 128 : (im1b1 >= -1 && im1b1 <= 1) ? im1b1 : (im2b1 >= -1 && im2b1 <= 1) ? im2b1 : 255
NDVI_VALUES_20180810T140829_20180813T140819.tif
['3_039.tif', '3_139.tif']
(im1b1 == 128 && im2b1 == 128) ? 128 : (im1b1 == 0 || im2b1 == 0) ? 0 : 255
NDVI_MASK_20180810T140829_20180813T140819.tif
['4_039.tif', '4_139.tif']
(im1b1 == 128 && im2b1 == 128) ? 128 : (im1b1 == 0 || im2b1 == 0) ? 0 : 255
MASK_20180810T140829_20180813T140819.tif
['5_0

### Previous output 

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.